In [256]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# import scipy
import numpy as np
# import pandas as pd
import os

from calcium import calcium
# from wheel import wheel
# from visualize import visualize
# from tqdm import trange

import scipy




Autosaving every 180 seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [276]:
#############################################
######### LOAD SUITE2P AND BINARIZE #########
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'
data_dir = '/media/cat/4TB/donato/nathalie/plane0'
#data_dir = '/media/cat/4TB/donato/renan/renan'
data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228'

# 
c = calcium.Calcium()
c.verbose = True             # outputs additional information during processing
c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = data_dir
c.load_suite2p()

# binarize fluorescence data
c.save_python = True         # save output as .npz file 
c.save_matlab = False         # save output as .mat file

###############################################
########## 3 parameters to tweak ##############
###############################################
#     1. Cutoff for calling somthing a spike:
#        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
#                                        higher -> less events; lower -> more events
#                                        start at default and increase if data is very noisy and getting too many noise-events
c.min_thresh_std_Fluorescence_onphase = 1.5
c.min_thresh_std_Fluorescence_upphase = 1.5

#     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
#        This is stored in high_cutoff: default 1.0
#                             - the lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces
c.high_cutoff = .5

#     3. Removing bleaching and drift artifacts using polynomial fits
#        This is stored in detrend_model_order
c.detrend_model_order = 5 # 1-5 polynomial fit

# 
#c.F=c.F[250:260]
c.binarize_fluorescence()

low pass filter:   0%|          | 0/1027 [00:00<?, ?it/s]

WARNING ***** Found cells with 0-[Ca] traces :  179
  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (1027, 92900)
         self.Fneu (neuropile):  (1027, 92900)
         self.iscell (Suite2p cell classifier output):  (1788, 2)
         # of good cells:  (1027,)
         self.spks (deconnvoved spikes):  (1027, 92900)
         self.stat (footprints structure):  (1027,)
         mean std over all cells :  23.5

  Binarization parameters: 
        low pass filter low cuttoff:  0.5 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_Fluorescence_onphase:  1.5
        min_thresh_std_Fluorescence_upphase:  1.5
        min_width_event_Fluorescence:  15
        min_width_event_oasis:  2
        min_event_amplitude:  1


model filter: remove bleaching or trends: 100%|██████████| 1027/1027 [00:13<00:00, 77.05it/s]
binarizing continuous traces filtered fluorescence onphase:   0%|          | 0/1027 [00:00<?, ?it/s]

WARNING ***** Found cells with 0-[Ca] traces :  179


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1027/1027 [00:15<00:00, 68.16it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1027/1027 [00:31<00:00, 32.94it/s]


   Oasis based binarization skipped by default ... 


In [195]:
#####################################################
############## LOAD INSCOPIX AND BINARIZE ###########
#####################################################

# input calcium traces .csv file from Inscopix
fnames = np.loadtxt('laurenz.txt',dtype='str')
for fname in fnames:
    #fname = '/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/DON-005099_20210609_TRACES_ACQ.csv'

    # 
    c = calcium.Calcium()
    c.verbose = True             # outputs additional information during processing
    c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
    c.data_dir = os.path.split(fname)[0]

    # Load inscopix data
    c.save_python = True         # save output as .npz file
    c.save_matlab = True         # save output as .mat file
    c.fname_inscopix = fname
    c.load_inscopix()

    #
    c.binarize_fluorescence()

low pass filter:  48%|████▊     | 98/203 [00:00<00:00, 976.06it/s]

cells:  (203, 36026)
F times:  (36026,)
         mean std over all cells :  4.5

  Binarization parameters: 
        low pass filter low cuttoff:  0.25 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_Fluorescence_onphase:  1.5
        min_thresh_std_Fluorescence_upphase:  1.5
        min_width_event_Fluorescence:  15
        min_width_event_oasis:  2
        min_event_amplitude:  1


model filter: remove linear trends up/down: 100%|██████████| 203/203 [00:00<00:00, 324.57it/s]
binarizing continuous traces filtered fluorescence onphase:  59%|█████▉    | 120/203 [00:00<00:00, 1189.85it/s]

 std global:  3.5


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 203/203 [00:00<00:00, 866.10it/s] 
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 203/203 [00:00<00:00, 826.36it/s] 


   Oasis based binarization skipped by default ... 


In [277]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
#fname = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'
fname = '/media/cat/4TB/donato/nathalie/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/renan/renan/binarized_traces.npz'
# 
fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'

c = calcium.Calcium()
c.fname = fname
c.load_binarization()

# pick a random cell to visuzlie
cell_id = 307

# 
scale = 100

####################################################
fig = plt.figure()
t=np.arange(c.F_filtered.shape[1])/c.sample_rate

if False:
    for k in range(800,848,1):
        plt.plot(t,c.F_filtered[k]/scale+k*1, linewidth=3, label='filtered', alpha=.8)
    
else:
    #plt.plot(t,c.F_onphase_bin[cell_id]*.9, linewidth=3, label='onphase', alpha=.8)
    #plt.plot(t,c.F_upphase_bin[cell_id], linewidth=3, label='upphase', alpha=.8)
    #plt.legend(fontsize=20)

    plt.plot(t,(c.F_filtered[cell_id]-np.median(c.F_filtered[cell_id]))/scale, linewidth=3, label='filtered', alpha=.8,
            c='black')
    plt.plot(t,c.F_processed[cell_id]/scale, linewidth=3, label='filtered', alpha=.8,
            c='blue')
    plt.plot(t,c.F_onphase_bin[cell_id]*.9, linewidth=3, label='onphase', alpha=.4,
            c='orange')
    plt.plot(t,c.F_upphase_bin[cell_id], linewidth=3, label='upphase', alpha=.4,
            c='green')
#plt.legend(fontsize=20)
plt.title("Cell: "+str(cell_id) + "\nSpike threshold: "+str(c.min_thresh_std_Fluorescence_upphase)+
          ", lowpass filter cutoff (hz): " +str(c.high_cutoff)+
          ", detrend polynomial model order: "+str(c.detrend_model_order))

plt.xlabel("Time (sec)")
plt.xlim(t[0], t[-1])
plt.show()

(1027, 92900)


In [279]:
######################################################
############# SHOW RASTERS ###########################
######################################################
c = calcium.Calcium()
c.fname = fname
c.load_binarization()

idx = np.where(c.F_upphase_bin==1)

# 
img = np.zeros((c.F_onphase_bin.shape[0], c.F_onphase_bin.shape[1]))
print (img.shape)
for k in range(idx[0].shape[0]):
    img[idx[0][k],idx[1][k]-5: idx[1][k]+5]=1

#
plt.figure()
plt.imshow(img, aspect='auto',
          cmap='Greys',
           extent=[0,img.shape[1]/c.sample_rate,
                  img.shape[0]-0.5,-0.5],
           
          interpolation='none')
plt.ylabel("Neuron ID (ordered by SNR by Suite2p)")
plt.xlabel("Time (sec)")
plt.title("Spike threshold: "+str(c.min_thresh_std_Fluorescence_upphase)+
          ", lowpass filter cutoff (hz): " +str(c.high_cutoff)+
          ", detrend polynomial model order: "+str(c.detrend_model_order))
plt.show()


(1027, 92900)
(1027, 92900)
